In [1]:
from typing import NamedTuple, Iterator

from tia.cov import (
    database_cursor,
    get_file_table,
    get_context_table,
    get_line_table,
    FileTableRow,
    ContextTableRow,
    LineTableRow,
)

In [2]:
Id = int
Ids = Iterator[Id]

FilePath = str
FilePaths = Iterator[FilePath]

TestName = str
TestNames = Iterator[TestName]


class CoverageMapSingle(NamedTuple):
    """
    File based coverage map (not line based w.r.t. production code functions)
    of single test name to potentially multiple production code files.
    """
    test: TestName
    production_code: FilePaths


CoverageMap = Iterator[CoverageMapSingle]
# File based coverage map (not line based) of test names to potentially multiple production code files per test name.


class ImpactMapSingle(NamedTuple):
    """
    File based impact map (not line based w.r.t. test code functions)
    of single production code file to potentially multiple test names
    """
    production_code: File
    tests: TestNames


ImpactMap = Iterator[ImpactMapSingle]
# File based impact map (not line based) of production code files to potentially multiple test names per production code file.

In [3]:
with database_cursor('../tests/data/.coverage') as cursor:
    production_file_list = [row for row in get_file_table(cursor)]
production_file_list

[FileTableRow(file_id=1, path='/home/fk/github/python-tia/tia/__init__.py'),
 FileTableRow(file_id=2, path='/home/fk/github/python-tia/tia/config.py'),
 FileTableRow(file_id=3, path='/home/fk/github/python-tia/tia/env.py')]

In [4]:
def get_file_ids(file_table: Iterator[FileTableRow], file_paths: FilePath) -> Ids:
    # return single element content of generator instead of whole generator
    for file_id, path in file_table:
        for file_path in file_paths:
            if file_path == path:
                yield file_id

In [5]:
with database_cursor('../tests/data/.coverage') as cursor:
    file_table_rows = get_file_table(cursor)
    file_id_list = [file_id for file_id in get_file_ids(file_table_rows, ['/home/fk/github/python-tia/tia/config.py', '/home/fk/github/python-tia/tia/env.py'])]
file_id_list

[2, 3]

In [6]:
# Making this Iterator[int] -> Iterator[Iterator[int]] probably no good idea.
def get_context_ids(line_table: Iterator[LineTableRow], prod_file_id: Id) -> Ids:
    already_seen_contexts = set()
    for file_id, context_id, _ in line_table:
        if file_id == prod_file_id:
            if context_id not in already_seen_contexts:
                already_seen_contexts.add(context_id)
                yield context_id

In [7]:
with database_cursor('../tests/data/.coverage') as cursor:
    #
    file_table_rows = get_file_table(cursor)
    production_file_ids = get_file_ids(file_table_rows, ['/home/fk/github/python-tia/tia/config.py', '/home/fk/github/python-tia/tia/env.py'])
    #
    for production_file_id in production_file_ids:
        line_table = get_line_table(cursor)
        context_ids = [id for id in get_context_ids(line_table, production_file_id)]
        print(context_ids)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 11, 12]


In [8]:
def get_test_name(context_table: Iterator[ContextTableRow], test_context_ids: Ids) -> TestNames:
    for context_id, context in context_table:
        for test_id in test_context_ids:
            if context_id == test_id:
                yield context

In [9]:
# impact mappping from changed production code files to impacted tests
changed_production_code = ['/home/fk/github/python-tia/tia/config.py', '/home/fk/github/python-tia/tia/env.py']
with database_cursor('../tests/data/.coverage') as cursor:

    file_table_rows = get_file_table(cursor)
    production_file_ids = get_file_ids(file_table_rows, changed_production_code)

    for production_file_id in production_file_ids:
        line_table = get_line_table(cursor)
        context_ids = [id for id in get_context_ids(line_table, production_file_id)]

        context_table = get_context_table(cursor)
        test_names = [test for test in get_test_name(context_table, context_ids)]
        print(changed_production_code)
        print(context_ids)
        print(test_names)
        # '' in test names due to ContextTableRow(context_id=1, context='') -> determined by coverage.py (dynamic context)

['/home/fk/github/python-tia/tia/config.py', '/home/fk/github/python-tia/tia/env.py']
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
['', 'test_reading_existing_valid_config_file_returns_string', 'test_reading_existing_invalid_config_file_raises_error', 'test_reading_non_existing_config_file_raises_exception', 'test_read_valid_parent_key_config', 'test_read_valid_explicit_full_blown_pipelines_config', 'test_read_valid_implicit_full_blown_pipelines_config', 'test_read_valid_single_pipeline_with_dirs_only_config', 'test_read_valid_single_pipeline_with_files_only_config', 'test_read_invalid_pipelines_config']
['/home/fk/github/python-tia/tia/config.py', '/home/fk/github/python-tia/tia/env.py']
[1, 11, 12]
['', 'test_is_some_ci', 'test_is_no_ci']
